<a href="https://colab.research.google.com/github/mandie021/big-data-challenge/blob/main/level-1/level_1_officeproducts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.8 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [2]:
#import postgres
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-02 19:40:36--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.31MB/s    in 0.7s    

2022-03-02 19:40:38 (1.31 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
#create session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Office_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
office_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Office_Products_v1_00.tsv.gz"), sep="\t", header=True)
office_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   43081963|R18RVCKGH1SSI9|B001BM2MAC|     307809868|Scotch Cushion Wr...| Office Products|          5|            0|          0|   N|                Y|          Five Stars|      Great product.| 2015-08-31|
|         US|   10951564|R3L4L6LW1PUOFY|B00DZYEXPQ|      75004341|Dust-Off Compress...| Office Products|          5|    

In [6]:
#Remove duplicates and na values
office_df = office_df.dropna()
office_df = office_df.dropDuplicates()
office_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [8]:
# make colummns int
office_df = office_df.withColumn("customer_id", office_df["customer_id"].cast(IntegerType()))
office_df = office_df.withColumn("product_parent", office_df["product_parent"].cast(IntegerType()))
office_df = office_df.withColumn("star_rating", office_df["star_rating"].cast(IntegerType()))
office_df = office_df.withColumn("helpful_votes", office_df["helpful_votes"].cast(IntegerType()))
office_df = office_df.withColumn("total_votes", office_df["total_votes"].cast(IntegerType()))

In [9]:
# Count the number of records (rows) in the dataset
office_df.count()

2642195

In [10]:
# Transform the dataset to fit the tables in the schema file
#imports and review id table
from pyspark.sql.functions import to_date

review_id_table = office_df.select(["review_id",  "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R100TJO6JFCAMT|   14928712|B004TU0IC2|     137501734| 2013-07-19|
|R100Y1XN4A4F4V|   12999038|B0055MI230|     305431275| 2015-06-19|
|R100YGDYQ4Z1D3|   19095535|B0039OWAZC|     110562620| 2012-07-31|
|R1028UG5FPCGDR|   20715457|B000IGMB5I|      76268412| 2013-05-04|
|R1031OV54Z62RO|   14690372|B008HBFADQ|     365473002| 2013-05-28|
|R103AODD96V5NG|   11159138|B00QSQ4JNO|     685671327| 2015-05-09|
|R103MJ18PCJHNC|   11207636|B0088VSXFM|     869192570| 2013-02-06|
|R103SAA2GS4XJS|   14844411|B001GAOTW8|     727986349| 2015-08-07|
|R104GO3BRW185C|    7060177|B00QGRJZNY|     236993214| 2015-04-29|
|R105AOVUVHOROM|   52857704|B0073W70SI|     830593102| 2012-09-15|
|R105R8IOXOPRJ6|    6658826|B00AW4GOXG|     264625908| 2013-12-13|
|R105SVH9NBHJ83|   23284325|B00GGZDXMK|     984764024| 2014-07

In [11]:
review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [12]:
# Transform the dataset to fit the tables in the schema file
#imports and products table
products = office_df.select(["product_id", "product_title"])
products = products.drop_duplicates(["product_id", "product_title"])
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00PITWBPA|Teacher Lesson Pl...|
|B0038ZTDGQ|        LINKYO TN650|
|B0028N6PSI|Mead Composition ...|
|B001IZX8AA|Generic DR400 and...|
|B010B1ROR4|Viz-pro Double-si...|
|B00WT9IDPE|AIM Compatible Re...|
|B003ZZUKPC|HTS 262B1 1,000 P...|
|B00NQ3UB34|Plush Cute Genuin...|
|B00JFOJ7PG|BIC Kids Pencil -...|
|B000AN1Q64|Pendaflex Hanging...|
|B00X5G94G2|Elmer's Freestyle...|
|B000BV0AYY|Canon PGI-5 BK 06...|
|B00SKO1E8A|No Trespassing We...|
|B00JXQR1ZY|KeySmart Extended...|
|B002OJN3VI|1 Pack. Refurbish...|
|B004OA6X6C|VTech LS6405 Acce...|
|B002FPQBEM|ENGLISH US LARGE ...|
|B002PIWE18|3 Mil 4x6 Photo L...|
|B000A2BJR6|Canon CLI-8 Ink Tank|
|B0006HVJ3Y|Highland Notes, Y...|
+----------+--------------------+
only showing top 20 rows



In [13]:
products.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [14]:
# Transform the dataset to fit the tables in the schema file
#imports and customers table
customers = office_df.groupby(["customer_id"]).count()
customers = customers.withColumnRenamed("count", "customer_count")
customers.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   43742707|             2|
|   22163154|             3|
|   15914833|             3|
|   17057106|             1|
|   38767673|             1|
|   47765241|             2|
|   30070564|             3|
|   11227090|             2|
|    4631901|             1|
|    1735665|             2|
|   43080114|             2|
|   52753379|             1|
|   21482558|             1|
|   19716806|             5|
|   23566763|             1|
|   19252780|             2|
|   11249570|             1|
|   46687387|             6|
|   52773179|            15|
|   47449696|             1|
+-----------+--------------+
only showing top 20 rows



In [15]:
#convert count to int
customers = customers.withColumn("customer_count", customers["customer_count"].cast(IntegerType()))

In [16]:
customers.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [17]:
# Transform the dataset to fit the tables in the schema file
#imports and vine table
vine_table = office_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100TJO6JFCAMT|          5|            0|          0|   N|
|R100Y1XN4A4F4V|          5|            0|          0|   N|
|R100YGDYQ4Z1D3|          4|            0|          0|   N|
|R1028UG5FPCGDR|          5|            0|          0|   N|
|R1031OV54Z62RO|          5|            1|          1|   N|
|R103AODD96V5NG|          5|            0|          0|   N|
|R103MJ18PCJHNC|          5|            0|          0|   N|
|R103SAA2GS4XJS|          5|            0|          0|   N|
|R104GO3BRW185C|          5|            0|          0|   N|
|R105AOVUVHOROM|          3|            2|          6|   N|
|R105R8IOXOPRJ6|          4|            1|          1|   N|
|R105SVH9NBHJ83|          5|            0|          0|   N|
|R1065Z3H0BDRIK|          5|            0|          0|   N|
|R1078RCFJDSZ1D|          5|            

In [18]:
vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [19]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://bigdatahwdb.crkggtvwjikn.us-east-1.rds.amazonaws.com:5432/hw_lvl1_db"
config = {"user":"root", 
          "password": "*****", 
          "driver":"org.postgresql.Driver"}

In [20]:
# Write DataFrame to review_id table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [21]:
# Write DataFrame to products table in RDS
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

Py4JJavaError: ignored

In [ ]:
# Write DataFrame to customers table in RDS
customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to vine_table table in RDS
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)